# Demo : using UNET to detect random squares in images

In this notebook, **we're going to build a simple UNET model** to detect squares with random color at random location in a black image.
It is a binary segmentation problem where pixels can only belong / not belong to a square.

Data are generated through a custom synthetic Dataset providing images of squares and associated masks. We then:

* `train the model` - fit the model on images of squares
* `test and evaluate` - test model prediction and compute sgmentation scores

The task at hand beeing simple, no data augmentation is used.

This simple use case can be modified to check quickly the implementation of a custom model architecture.  


In [ ]:
import logging

import torch

from src.model.unet import UNetModel
from src.dataset.square_dataset import SquareDataset
from src.benchmark.train import train, HyperParameters, EarlyStoppingParams, TrainParameters

# algorithm hyperparameters
# -------------------------
NB_EPOCHS = 40
BATCH_SIZE = 16
LEARNING_RATE = 0.0005

# model hyperparameters
# ---------------------
# for the simple task at hand the base number (12) used to compute the number of feature maps
# in each model layer can be low. In the original paper this value is 64
BASE_FM_NUMBER = 12  
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_NB_CHANNELS = 3 # RGB images
NB_LABELS = 1       # only one foreground class, remaining pixels beeing background 
INIT_WEIGHTS_PATH = None

MODEL_PATH = "./square_model_dict.pth"

logging.basicConfig(filename=None, filemode="w", level=logging.DEBUG)

# select device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Selected device : {device}")


In [ ]:
""" Train the model on a synthetic dataset composed of squares with different colors (1-label problem) """

# build model architecture
model = UNetModel(IMG_NB_CHANNELS, NB_LABELS, BASE_FM_NUMBER)

# create train and validation datasets of 100 images each
nb_images = 100
train_dataset = SquareDataset(IMG_NB_CHANNELS, IMG_WIDTH, IMG_HEIGHT, nb_images, 20, 80)
validation_dataset = SquareDataset(IMG_NB_CHANNELS, IMG_WIDTH, IMG_HEIGHT, nb_images, 20, 80)

# set training hyperparameters
hyperparameters = HyperParameters(
    NB_EPOCHS, BATCH_SIZE, LEARNING_RATE, EarlyStoppingParams(3, 0.001)
)

# run the training process
train(
    model,
    train_dataset,
    validation_dataset,
    TrainParameters(hyperparameters, INIT_WEIGHTS_PATH, device),
    MODEL_PATH,
)


In [ ]:
""" test the model on a synthetic dataset composed of squares with different colors (1-label problem)"""
import os
from pathlib import Path
from src.benchmark.test import test, TestMetrics, TestReport

# build model architecture and load weights generated during the training process
model = UNetModel(IMG_NB_CHANNELS, 1, BASE_FM_NUMBER).to(device)
model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))

# create a test dataset of 10 images
nb_images = 10 
test_dataset = SquareDataset(IMG_NB_CHANNELS, IMG_WIDTH, IMG_HEIGHT, nb_images, 20, 80)

# Configure test report : we are only interested here in micro-averaged accuracy
test_report = TestReport(Path(os.getcwd()).joinpath("square_dataset_report.json"),[TestMetrics.ACCURACY])

# run evaluation (with segmentation image display activated : verbise = 1)
test(device, model, test_dataset, test_report, verbose=1)
